In [ ]:
import math
import pandas as pd
import geojson as json
from shapely.geometry import Point, LineString

In [ ]:
with open('Bloomingdale_Trail.geojson') as f:
    trail_json = json.load(f)
trail = LineString(trail_json['features'][0]['geometry']['coordinates'])

crime_loc = pd.read_csv("chicago_crimes_subset.csv")

In [ ]:
def get_point(obs):
    return Point(obs['Longitude'], obs['Latitude'])
loc = crime_loc.apply(get_point, axis = 1)

In [ ]:
def get_distance(obs):
    # calculate Euclidean distance in degrees
    return obs.distance(trail)
crime_loc['Distance'] = loc.apply(get_distance)

In [ ]:
def get_coords(pt):
    return (math.radians(pt.y), math.radians(pt.x))

def get_direction(obs):
    ## calculate compass bearing between obs, nearest point on trail
    pt_lat, pt_lon = get_coords(obs)
    trail_point = trail.interpolate(trail.project(obs))
    trail_lat, trail_lon = get_coords(trail_point)
    
    diff_lon = pt_lon - trail_lon
    x = math.sin(diff_lon) * math.sin(pt_lat)
    y = math.cos(trail_lat) * math.sin(pt_lat) - (math.sin(trail_lat) * math.cos(pt_lat) * math.cos(diff_lon))
    init_bearing = math.degrees(math.atan2(x, y))
    compass_bearing = (init_bearing + 360) % 360
    
    if compass_bearing > 90 and compass_bearing <= 270:
        return 'S'
    return 'N'

crime_loc['Direction'] = loc.apply(get_direction)

In [ ]:
crime_loc.to_csv('chicago_crimes_subset_distance.csv')